<a href="https://colab.research.google.com/github/TusKy-M/Handwritten-Eqn-Solve/blob/main/DCCNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install q keras==2.3.1

     |████████████████████████████████| 378kB 16.1MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow

In [ ]:
print(tensorflow.__version__)

1.15.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/My\ Drive/Datasets/extracted_images.zip

Streaming output truncated to the last 5000 lines.
 extracting: extracted_images/y/exp69885.jpg  
 extracting: extracted_images/y/exp69903.jpg  
  inflating: extracted_images/y/exp69910.jpg  
 extracting: extracted_images/y/exp69911.jpg  
 extracting: extracted_images/y/exp69925.jpg  
 extracting: extracted_images/y/exp69927.jpg  
 extracting: extracted_images/y/exp69937.jpg  
 extracting: extracted_images/y/exp69948.jpg  
 extracting: extracted_images/y/exp69951.jpg  
  inflating: extracted_images/y/exp69997.jpg  
 extracting: extracted_images/y/exp70.jpg  
 extracting: extracted_images/y/exp70003.jpg  
 extracting: extracted_images/y/exp70103.jpg  
  inflating: extracted_images/y/exp70158.jpg  
 extracting: extracted_images/y/exp70164.jpg  
 extracting: extracted_images/y/exp70171.jpg  
 extracting: extracted_images/y/exp70176.jpg  
 extracting: extracted_images/y/exp70179.jpg  
 extracting: extracted_images/y/exp70200.jpg  
  inflating: extracted_images/y/exp70203.jpg  
  inflating:

**Data preprocessing**

In [ ]:
import os
import cv2
import numpy as np
import math
from scipy import ndimage
def load_images_from_folder(folder):
    train_data=[]
    k =0
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename),cv2.IMREAD_GRAYSCALE)
        if img is not None:
            if k>4000:
              break
            img = ~img
            kernel = np.ones((3,3),np.uint8)
            dilation = cv2.dilate(img,kernel,iterations = 1)
            #erosion = cv2.erode(dilation,kernel,iterations = 1)
            blur = cv2.GaussianBlur(dilation,(3,3),1)
            ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            gray = cv2.resize(th3, (28, 28))
            while np.sum(gray[0]) == 0:
                gray = gray[1:]

            while np.sum(gray[:,0]) == 0:
                gray = np.delete(gray,0,1)

            while np.sum(gray[-1]) == 0:
                gray = gray[:-1]

            while np.sum(gray[:,-1]) == 0:
                gray = np.delete(gray,-1,1)

            rows,cols = gray.shape
            if rows > cols:
                factor = 20.0/rows
                rows = 20
                cols = int(round(cols*factor))
                gray = cv2.resize(gray, (cols,rows))
            else:
                factor = 20.0/cols
                cols = 20
                rows = int(round(rows*factor))
                gray = cv2.resize(gray, (cols, rows))
            colsPadding = (int(math.ceil((28-cols)/2.0)),int(math.floor((28-cols)/2.0)))
            rowsPadding = (int(math.ceil((28-rows)/2.0)),int(math.floor((28-rows)/2.0)))
            gray = np.lib.pad(gray,(rowsPadding,colsPadding),'constant')
            def getBestShift(img):
                cy,cx = ndimage.measurements.center_of_mass(img)

                rows,cols = img.shape
                shiftx = np.round(cols/2.0-cx).astype(int)
                shifty = np.round(rows/2.0-cy).astype(int)

                return shiftx,shifty
            def shift(img,sx,sy):
                rows,cols = img.shape
                M = np.float32([[1,0,sx],[0,1,sy]])
                shifted = cv2.warpAffine(img,M,(cols,rows))
                return shifted
            shiftx,shifty = getBestShift(gray)
            shifted = shift(gray,shiftx,shifty)
            gray = shifted
            cv2.imwrite("/content/final datasets/13/"+str(k)+".jpg", gray)
            k=k+1

In [ ]:
load_images_from_folder('/content/extracted_images/(')

In [ ]:
# 10 +; 11 -; 12 x; 13 y; 14 =

In [ ]:
img = cv2.imread('/content/final datasets/10/400.jpg')

In [ ]:
#img

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(img)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
figure(figsize=(10,10))
#ax1 = plt.subplot(2,len(ima),cnt)
plt.imshow(img,cmap="gray")
#plt.imshow(erosion,cmap="gray")

NameError: ignored

<Figure size 432x288 with 0 Axes>

In [ ]:
train_dir = "/content/final datasets/"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

     
train_datagen = ImageDataGenerator(rescale=1./255,
    
    data_format='channels_first',
    validation_split=0.2) 

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(28, 28),
    color_mode = "grayscale",
    batch_size=20,
    shuffle = True,
    classes = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14'],# 10 +; 11 -; 12 x; 13 y; 14 =
    class_mode="sparse",
    subset='training') 

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(28, 28),
    color_mode = "grayscale",
    batch_size=20,
    shuffle = True,
    classes = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14'],
    class_mode="sparse",
    subset='validation')

Found 45325 images belonging to 15 classes.
Found 11326 images belonging to 15 classes.


In [ ]:
import keras
keras.backend.set_image_data_format('channels_first')

**Training model**

In [ ]:
from keras.layers import merge, Input, concatenate
from keras.models import Model
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import MaxPooling2D, Convolution2D
img_rows, img_cols = 28, 28

nb_filters_1 = 64
nb_filters_2 = 128
nb_filters_3 = 256
nb_conv = 3
nb_conv_mid = 4
nb_conv_init = 5

init = Input(shape=(1, 28, 28),)

fork11 = Convolution2D(nb_filters_1, nb_conv_init, nb_conv_init,  activation="relu", border_mode='same')(init)
fork12 = Convolution2D(nb_filters_1, nb_conv_init, nb_conv_init, activation="relu", border_mode='same')(init)
merge1 = concatenate([fork11, fork12], axis=1, name='merge1')
# concat_feat = concatenate([concat_feat, x], mode='concat', axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))
maxpool1 = MaxPooling2D(strides=(2,2), border_mode='same')(merge1)

fork21 = Convolution2D(nb_filters_2, nb_conv_mid, nb_conv_mid, activation="relu", border_mode='same')(maxpool1)
fork22 = Convolution2D(nb_filters_2, nb_conv_mid, nb_conv_mid, activation="relu", border_mode='same')(maxpool1)
merge2 = concatenate([fork21, fork22, ], axis=1, name='merge2')
maxpool2 = MaxPooling2D(strides=(2,2), border_mode='same')(merge2)

fork31 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same')(maxpool2)
fork32 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same')(maxpool2)
fork33 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same')(maxpool2)
fork34 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same')(maxpool2)
fork35 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same')(maxpool2)
fork36 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", border_mode='same')(maxpool2)
merge3 = concatenate([fork31, fork32, fork33, fork34, fork35, fork36, ], axis=1, name='merge3')
maxpool3 = MaxPooling2D(strides=(2,2), border_mode='same')(merge3)

dropout = Dropout(0.5)(maxpool3)

flatten = Flatten()(dropout)
output = Dense(15, activation="softmax")(flatten)

model = Model(input=init, output=output)

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1, 28, 28)    0                                            
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 64, 28, 28)   1664        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 64, 28, 28)   1664        input_3[0][0]                    
__________________________________________________________________________________________________
merge1 (Concatenate)            (None, 128, 28, 28)  0           conv2d_21[0][0]                  
                                                                 conv2d_22[0][0]            

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), activation="relu", padding="same")`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(strides=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (4, 4), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (4, 4), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarn

In [ ]:
from keras import optimizers
ada = keras.optimizers.Adadelta(learning_rate=1.0, rho=0.95)
model.compile(optimizer=ada,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit_generator(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=100,
                              validation_steps=100,
                              epochs=20)

Epoch 1/20
100/100 [==============================] - 3s 30ms/step - loss: 1.1345 - accuracy: 0.6490 - val_loss: 0.1339 - val_accuracy: 0.9195
Epoch 2/20
100/100 [==============================] - 2s 24ms/step - loss: 0.3411 - accuracy: 0.8940 - val_loss: 0.5279 - val_accuracy: 0.9225
Epoch 3/20
100/100 [==============================] - 2s 24ms/step - loss: 0.3084 - accuracy: 0.9120 - val_loss: 0.1951 - val_accuracy: 0.9250
Epoch 4/20
100/100 [==============================] - 2s 25ms/step - loss: 0.1993 - accuracy: 0.9385 - val_loss: 0.0962 - val_accuracy: 0.9170
Epoch 5/20
100/100 [==============================] - 2s 24ms/step - loss: 0.1926 - accuracy: 0.9355 - val_loss: 0.0385 - val_accuracy: 0.9680
Epoch 6/20
100/100 [==============================] - 2s 25ms/step - loss: 0.1766 - accuracy: 0.9490 - val_loss: 0.0788 - val_accuracy: 0.9718
Epoch 7/20
100/100 [==============================] - 2s 24ms/step - loss: 0.2026 - accuracy: 0.9485 - val_loss: 0.1685 - val_accuracy: 0.9650